In [1]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 4096*2 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 03-18 00:53:10 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.6: Fast Llama patching. Transformers: 4.49.0. vLLM: 0.7.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.542 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 58.92%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 23.54 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 8192. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.54 GB. Also swap space = 6 GB.
INFO 03-18 00:53:19 config.py:549] This model supports multiple tasks: {'generate', 'reward', 'c

[W318 00:53:20.632400570 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 03-18 00:53:21 weight_utils.py:254] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-18 00:53:23 model_runner.py:1115] Loading model weights took 5.5976 GB
INFO 03-18 00:53:23 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-18 00:53:25 worker.py:267] Memory profiling takes 1.67 seconds
INFO 03-18 00:53:25 worker.py:267] the current vLLM instance can use total_gpu_memory (23.54GiB) x gpu_memory_utilization (0.59) = 13.87GiB
INFO 03-18 00:53:25 worker.py:267] model weights take 5.60GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 0.96GiB; the rest of the memory reserved for KV Cache is 7.24GiB.
INFO 03-18 00:53:25 executor_base.py:111] # cuda blocks: 3704, # CPU blocks: 3072
INFO 03-18 00:53:25 executor_base.py:116] Maximum concurrency for 8192 tokens per request: 7.23x
INFO 03-18 00:53:29 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occ

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:18<00:00,  1.47it/s]

INFO 03-18 00:53:47 model_runner.py:1562] Graph capturing finished in 18 secs, took 0.66 GiB
INFO 03-18 00:53:47 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 24.22 seconds



Unsloth 2025.3.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
import re
from datasets import load_dataset, Dataset
from search_module import search, get_question_answer, get_question_count
from rl_helpers import get_qa_dataset

train_dataset, test_dataset = get_qa_dataset()

Loading FAISS index from: /root/AutoDidact/faiss_index
Successfully loaded FAISS index
Loading chunks from: /root/AutoDidact/saved_data/chunks.pkl
Loading questions from: /root/AutoDidact/saved_data/questions.json
Successfully loaded 341 chunks and 676 questions


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [4]:
import os
os.environ["WANDB_PROJECT"] = "bootstrap-search-rl"

In [5]:
# from UnslothGRPOTrainerTemp import UnslothGRPOConfig, _UnslothGRPOTrainer
import UnslothGRPOTrainerTemp
training_args = UnslothGRPOTrainerTemp.UnslothGRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    use_agentic_generate = True, # use agentic generation
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 1024,
    max_completion_length = 1024,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 101,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "full_local_training",

)

In [6]:


import rl_helpers
# importlib.reload(rl_helpers)


def agentic_generate(
        prompts:list[str],
        generate_fn,
        max_generations:int=6,
        ):
    return run_agent(generate_fn, tokenizer, prompts, max_generations)
model.agentic_generate = agentic_generate


from vllm import SamplingParams
verifier_sampling_params = SamplingParams(
    temperature = 0.1,
    top_p = 0.95,
    max_tokens = 4096,
)
def verifier_generate_fn(inputs):
    return model.fast_generate(
        inputs,
        sampling_params = verifier_sampling_params,
    )


run_agent = rl_helpers.run_agent
reward_correctness = rl_helpers.build_reward_correctness_fn(verifier_generate_fn, tokenizer,)
reward_formatting = rl_helpers.reward_formatting

import UnslothGRPOTrainerTemp
trainer = UnslothGRPOTrainerTemp.UnslothGRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        reward_correctness,
        reward_formatting,
    ],
    args = training_args,
    train_dataset = train_dataset,
)

In [7]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 608 | Num Epochs = 1 | Total steps = 101
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 167,772,160/4,796,452,864 (3.50% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure 

['What was the reason for the master caution and warning alarm at 38 hours during the Apollo 13 mission?', 'What was the reason for the master caution and warning alarm at 38 hours during the Apollo 13 mission?', 'What was the reason for the master caution and warning alarm at 38 hours during the Apollo 13 mission?', 'What was the reason for the master caution and warning alarm at 38 hours during the Apollo 13 mission?', 'What was the reason for the master caution and warning alarm at 38 hours during the Apollo 13 mission?', 'What was the reason for the master caution and warning alarm at 38 hours during the Apollo 13 mission?', 'What was the reason for the master caution and warning alarm at 38 hours during the Apollo 13 mission?', 'What was the reason for the master caution and warning alarm at 38 hours during the Apollo 13 mission?']


/root/AutoDidact/UnslothGRPOTrainerTemp.py:943: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  completion_ids = [torch.tensor(ids, device=device) for ids in completion_ids]
Processed prompts: 100%|██████████| 8/8 [00:00<00:00, 37.67it/s, est. speed input: 5686.72 toks/s, output: 75.38 toks/s]


rewards_per_func: tensor([0.3750, 0.3500], device='cuda:0')


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / reward_correctness,rewards / reward_formatting
1,-0.000000,0.725000,0.841342,158.375000,0.000000,0.375000,0.350000
2,0.000000,0.212500,0.601041,196.875000,0.000000,0.125000,0.087500
3,0.000000,0.900000,0.723089,212.125000,0.000496,0.375000,0.525000
4,0.000000,0.437500,0.362284,195.500000,0.000650,0.000000,0.437500
5,0.000000,0.350000,0.374166,252.250000,0.000427,0.000000,0.350000
6,0.000000,0.437500,0.362284,290.625000,0.000602,0.000000,0.437500
7,0.000000,0.437500,0.362284,238.750000,0.000577,0.000000,0.437500
8,0.000000,0.350000,0.374166,159.500000,0.000379,0.000000,0.350000
9,0.000000,0.562500,0.575543,227.875000,0.000637,0.125000,0.437500
10,0.000000,0.775000,0.647523,349.750000,0.000890,0.250000,0.525000


...

Processed prompts: 100%|██████████| 8/8 [00:00<00:00, 54.31it/s, est. speed input: 7801.08 toks/s, output: 108.72 toks/s]


rewards_per_func: tensor([1.0000, 0.7000], device='cuda:0')
['What was the reason for the first abort in the Apollo program?', 'What was the reason for the first abort in the Apollo program?', 'What was the reason for the first abort in the Apollo program?', 'What was the reason for the first abort in the Apollo program?', 'What was the reason for the first abort in the Apollo program?', 'What was the reason for the first abort in the Apollo program?', 'What was the reason for the first abort in the Apollo program?', 'What was the reason for the first abort in the Apollo program?']


/root/AutoDidact/UnslothGRPOTrainerTemp.py:943: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  completion_ids = [torch.tensor(ids, device=device) for ids in completion_ids]
Processed prompts: 100%|██████████| 8/8 [00:00<00:00, 40.77it/s, est. speed input: 8338.94 toks/s, output: 81.60 toks/s]


rewards_per_func: tensor([0.0000, 0.7000], device='cuda:0')


TrainOutput(global_step=101, training_loss=0.00014615571892236538, metrics={'train_runtime': 2771.7194, 'train_samples_per_second': 0.292, 'train_steps_per_second': 0.036, 'total_flos': 0.0, 'train_loss': 0.00014615571892236538})

<a name="Inference"></a>
### Inference
Now let's try benchmark the model we trained!

In [8]:
from vllm import SamplingParams
import rl_helpers
sampling_params = SamplingParams(
    temperature = 0.5,
    top_p = 0.95,
    max_tokens = 4096,
)

def eval_generate_fn(inputs):
    return model.fast_generate(
        inputs,
        sampling_params = sampling_params,
        lora_request = model.load_lora("full_local_training/checkpoint-101"), # load the trained LoRA
    )


rl_helpers.run_eval(
    generate_fn=eval_generate_fn,
    verify_fn=reward_correctness,
    tokenizer=tokenizer,
)

Processed prompts:   0%|          | 0/68 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 68/68 [00:01<00:00, 38.50it/s, est. speed input: 7604.52 toks/s, output: 109.34 toks/s]

RESULTS:
percentage of correct answers: 0.5882352941176471


[{'messages': [{'role': 'system',
    'content': 'Cutting Knowledge Date: December 2023\nToday Date: 18 Mar 2025\n\nWhen you receive a tool call response, use the output to format an answer to the original user question.\n\nYou are a helpful assistant with tool calling capabilities.\n'},
   {'role': 'user',
    'content': 'You are a research assistant, and you use the search_corpus tool to find answers to questions.\nGiven a question, answer it using by doing searches using the search_corpus tool.\nTo use the search_corpus tool, respond with a JSON for a function call with its proper arguments.\n\nYou may also reason in any message, thinking step by step about how to answer the question. Wrap your reasoning in <reasoning> and </reasoning> tags.\n\n{\n  "type": "function",\n  "function": {\n    "name": "search_corpus",\n    "description": "Search over the knowledge corpus with a given query",\n    "parameters": {\n      "type": "object",\n      "properties": {\n        "query": {\n     

In [10]:
# eval w/o lora
def eval_generate_fn(inputs):
    return model.fast_generate(
        inputs,
        sampling_params = sampling_params,
    )


rl_helpers.run_eval(
    generate_fn=eval_generate_fn,
    verify_fn=reward_correctness,
    tokenizer=tokenizer,
)

Processed prompts: 100%|██████████| 68/68 [00:01<00:00, 39.85it/s, est. speed input: 6465.53 toks/s, output: 158.31 toks/s]

RESULTS:
percentage of correct answers: 0.23529411764705882


[{'messages': [{'role': 'system',
    'content': 'Cutting Knowledge Date: December 2023\nToday Date: 18 Mar 2025\n\nWhen you receive a tool call response, use the output to format an answer to the original user question.\n\nYou are a helpful assistant with tool calling capabilities.\n'},
   {'role': 'user',
    'content': 'You are a research assistant, and you use the search_corpus tool to find answers to questions.\nGiven a question, answer it using by doing searches using the search_corpus tool.\nTo use the search_corpus tool, respond with a JSON for a function call with its proper arguments.\n\nYou may also reason in any message, thinking step by step about how to answer the question. Wrap your reasoning in <reasoning> and </reasoning> tags.\n\n{\n  "type": "function",\n  "function": {\n    "name": "search_corpus",\n    "description": "Search over the knowledge corpus with a given query",\n    "parameters": {\n      "type": "object",\n      "properties": {\n        "query": {\n     